# Python for (open) Neuroscience

_Lecture 0.4_ - Classes and objects

Luigi Petrucco

Jean-Charles Mariani

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vigji/python-cimec-2024/blob/main/lectures/Lecture0.4_Classes-objects.ipynb)

### Functions - recap

### Anatomy of a function

<p align="center">
  <img src="./files/function_anatomy.png" />
</p>

In [58]:
def print_args(a, b, c=3):
    print(f"a={a}, b={b}, c={c}")


print_args(1, 2, 5)

a=1, b=2, c=5


### The scope of function variables

`scope` of a variable is the technical term to indicate the parts of your code where that variable will be known to the interpreter

Variables defined inside the function will live only during the execution of the function!

In [68]:
def sum_vals(a, b):
    local_sum_variable = a + b
    return local_sum_variable


a = 1
b = 2
d = sum_vals(a, b)

# this will fail, as we are out of the scope for this variable 
# that will live and die during the function execution:
print(local_sum_variable)

NameError: name 'local_sum_variable' is not defined

A resource that you can use to explore scopes and variable assignments on and out from functions is [Python tutor](https://pythontutor.com/render.html#mode=display), a tool that helps you display the flow of small snippets of python code. Try it out! (but don't worry if it looks confusing - it can be a bit advanced)

On the other side, functions can have access to variables defined outside the function. However, we should not use them in the function code!

In [71]:
def sum_vals(a, b):
    # This will not fail. However, it should be avoided in general! it makes running the function
    # implicitely relying on code in the rest script.
    print(a_var_not_passed.pop())
    return a + b


a_var_not_passed = [1, 2]
a = 1
b = 2
d = sum_vals(a, b)
print(a_var_not_passed)

2
[1]


## Side effects

**Avoid sides effects!** Do not modify the passed values or variables existing ouside the function scope. In the cases you do it, do it consciously and make it very explicit in the name of the function or in the docstring!

In [72]:
# Here we will have troubles: the code modifies the list by popping out elements.
# A possible solution would be making a copy inside the function 
# (or write differently the code, but this is just an example)

a_list = [1, 2, 3]

def take_last(a_list):
    last = a_list.pop()  # this affects the original list

    # last = a_list.copy().pop()  # this instead would fix the issue

    return last


print(take_last(a_list))
print(a_list)

3
[1, 2]


In [80]:
# Sometimes you want to change input variable (eg, when you don't want 
# to duplicate large datasets in memory). In those casesm functions
# should be very explicitly indicating this way of operating on variables:
# "inplace" is the word we use for modifications to a variable passed to a function:

def double_list_inplace(input_list):
    """This function doubles values inside a list INPLACE.
    """
    for index, value in enumerate(input_list):
        input_list[index] = value*2
        
    # Note that I do not need to return any value here: I'm changing input_list directly!
        
str_list = [1, 10, 23]
print(str_list)
double_list_inplace(str_list)
print(str_list)  # the same list will contain different values now

[1, 10, 23]
[2, 20, 46]


# Imported functions

Many times you will import functions from external modules, like `random`. the functions docs give you all the information you need about how to use the function:

In [6]:
import random

?random.randint

(Practical 0.4.0)

## Classes and objects

Deep down the rabbit (Python) hole...

<p align="center">
  <img src="./files/class_structure.jpeg" />
</p>

(credits: Heather)

## Object-oriented programming (OOP)

**Disclaimer 0:** 

OOP can be confusing at first. You're entering the realm of True Programming Problems! Some concepts might feel a bit metaphysical.

**Disclaimer 1:**

This lecture is aimed more at understanding/reading existing code than at implementation. You won't probably write a lot of classes right now, but the concepts are very important to understand Python code!

OOP is a programming paradigm based on the concept of <span style="color:indianred">objects</span>, entities that _bind together data and operations on those data_.

## An example

Let's start with an example...

Imagine the following scenario:

 - You have an experiment, containing data, info on subjects, details on experimental stimulus, etc.
 - You want to implement some functions that operate on this dataset
 

**First option**: one entity -> many variables, many functions:

In [ ]:
# A bunch of variables referring to the same experiment (not defined for real here):
sampling_frequency = ...
imaging_data = ...
stimulus_data = ...

# A bunch of functions operating on those variables (not defined for real here):
resample(imaging_data, sampling_frequency, new_frequency=...)
crop_on_stimulus(imaging_data, stimulus_data, sampling_frequency, padding=...)

But: many variables around, even though we always refer to a single entity

**Second option**: one entity -> one variable, many functions:

In [ ]:
# A dictionary with bunch of keys referring to the same experiment (not defined for real here):
data_dictionary = {"sampling_frequency": ...,
                   "imaging_data": ...,
                   "stimulus_data": ...}

# A bunch of functions operating on this dictionary (not defined for real here):
resample(data_dictionary, new_frequency=...)
crop_on_stimulus(data_dictionary, padding=...)
...

But: notice how we have some functions that always require a given variable to run (`data_dictionary`).

It could be useful to represent together data that refer to some entity, and operations that we can do on them!

## \*Enter objects\*

Objects are entities that keep together:

 - <span style="color:indianred">Attributes</span>: similar to variables, kept together in an object as they would be in a dictionary

 - <span style="color:indianred">Methods</span>: similar to functions, but operating on all the attributes of an object (potentially taking more inputs)

Advantages of OOP:

 - represent together data and procedures operating on them
 - flexible data interface ("A dictionary on steroids")
 - (advanced: nicely define what to expose and what to keep private (abstraction - more on this at the end))

## An example

Let's look at an example: an object keeping together experimental metadata info and loading functions

In [120]:
# You don't need to look at code in this cell for now! (Here we are defining the class for our object)
class ParticipantData:
    """Represent data from an experimental subject.
    
    Methods:
    =======
    
    get_data_path: ...
    
    get_exp_data: ...
    
    get_exp_metadata: ...
    
    Attributes:
    ===========
    ...
    
    """
    def __init__(self, participant_id, age, condition, base_path="/path/to/data"):
        self.participant_id = participant_id
        self.age = age
        self.condition = condition
        self.base_path = base_path
        self.experiment_name = "Experiment 1"
    
    def get_data_path(self):
        """Generates a file path for participant data based on their ID and condition."""
        return f"{self.base_path}/subject_{self.participant_id}_{self.condition}.csv"
    
    def get_exp_metadata(self):
        """Return participant matadata."""
        return {
            "id": self.participant_id,
            "age": self.age,
            "condition": self.condition,
        }
    
    def get_exp_data(self):
        """Loads data for the participant from a file."""
        data_path = self.get_data_path()
        print(f"Loading data from {data_path}...")  # We are not really loading here
        return [1, 2, 3]  # Dummy measurements

In [121]:
# Creation of a ParticipantData object: we pass relevant arguments in round brackets,
# as when calling functions. This operation is called "instantiation":
participant_a_data = ParticipantData(participant_id="A", age=28, condition="treatment")

### Attributes

In [105]:
# This object has a bunch of attributes; in this case, participant_id, age, condition, experiment_name
# (Note that objects can have attributes that we did not pass in the instantiation; 
# in this case, experiment_name)

# We access them using the `object_name.attribute_name` syntax:
print(participant_a_data.age)

print(participant_a_data.experiment_name)

28
Experiment 1


Accessing attributes is conceptually quite similar to accessing a dictionary, just with a different syntax (`object_name.attribute_name` instead of `dictionary_name["attribute_name"]`)!

Indeed, we can actually have a look at all attributes of an object with this trick:

In [106]:
# There is (most of the times) a .__dict__ attribute containing all others!
participant_a_data.__dict__

{'participant_id': 'A',
 'age': 28,
 'condition': 'treatment',
 'base_path': '/path/to/data',
 'experiment_name': 'Experiment 1'}

### Methods

Methods are functions that are called directly from objects and operate upon their values.

You can generally read about methods in the object documentation (accessible with the usual `?` syntax)

In [122]:
?participant_a_data

In [123]:
participant_a_data.get_data_path()  # get location of subject data

'/path/to/data/subject_A_treatment.csv'

In [115]:
data = participant_a_data.get_exp_data()  # load subject data

Loading data from /path/to/data/subject_A_treatment.csv...


Many times with methods we don't need to pass arguments! They already have all the attributes of the object to operate on.

### Variables as objects

We have already introduced the concept of methods: e.g., with lists and strings:

In [71]:
a_string = "Some text"

a_string.split()  # This is a method! It operates on the text stored in `a_string`


['Some', 'text']

### Classes and objects

To use objects, we first have to define classes.

- classes are the abstract definition of categories from which we create specific instances

- individual, separate objects are then created from a class

- A class stands to objects in the way the concept of a chair stands to real-world chairs that you see around, with their specific material, number of legs, etc.

For example, using the same "template" for subject data above (the class), we can produce different objects for each one of multiple subjects. Objects will have the same methods, and same attributes with different values:

In [124]:
# An object of the `ParticipantData` class:
participant_a_data = ParticipantData(participant_id="A", age=28, condition="treatment")

# A different object of the same `ParticipantData` class:
participant_b_data = ParticipantData(participant_id="B", age=35, condition="control")

## Objects and types in Python

You were not aware of this, but you're already a proficient user of objects!

In fact, in Python all variables are objects of different classes (the data types). IF you remember, you were calling  methods many times!

In [125]:
text = "Some text"  # this is an object of "str" class!
text.split()  # this is simply a method!

['Some', 'text']

In [129]:
a_int = 1
a_int.to_bytes()  # even integers actually have methods!

b'\x01'

(Advanced: if you play around with standard types, you'll see that they do not really have attributes as we described above, only methods; this is because they are implemented in a way that keep the user away from accessing the data represented in the variable to avoid screw-ups. If you're curious let's discuss in practicals time!)

Practical 0.4.1

### [optional] Inspect objects with `dir()`

We can check out attributes and methods of a class with the base Python function `dir`

In [130]:
participant_a_data = ParticipantData(participant_id="A", age=28, condition="treatment")

dir(participant_a_data)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'age',
 'base_path',
 'condition',
 'experiment_name',
 'get_data_path',
 'get_exp_data',
 'get_exp_metadata',
 'participant_id']

### `callable()`

We can verify what is callable and what is not using the `callable()` function:

In [ ]:
the_run_method = getattr(my_horse, "run")
print("run method: ", callable(the_run_method))

the_name_attribute = getattr(my_horse, "name")
print("name attribute: ", callable(the_name_attribute))